# observations

1. There were almost the exact same ratio of repeat buyers for both men and women players(73% of Men and Women made more than one purchase) which says that gender doesn't play a part in whether someone makes a repeat purchase. 
2. Age group of under 10 was the least likely to make a repeat purchase (3% of under 10 made a repeat purchase)  while (20-24) were the most likley (77%). 
3. Looking at the normalized totals, men were expected to spend more money on game items than women and people over 40 were expected to spend the higest amount. Men over 40 years will spend the most on the game whereas a female between the ages of 20-24 might be expected to spend the least. 

In [226]:
import pandas as pd
import numpy as np

df = pd.read_json('/Users/hanna/Downloads/UDEN201805DATA1-master/Week4/PandasHW/HeroesOfPymoli/purchase_data.json')



# Total Number of Players

In [227]:
total_players = df.groupby('SN').count().shape[0]
total_num_players = pd.DataFrame({
    'Total Players' : [total_players],
})
total_num_players

,Total Players
0,573


# Purchasing Analysis 

In [228]:

purchasing_analysis = pd.DataFrame({
    'Number of Unique Items': [num_unique_items],
    'Average Price' : [avg_price],
    'Total Purchases': [total_purchases],
    'Total Revenue' : [total_revenue],
})

purchasing_analysis



,Average Price,Number of Unique Items,Total Purchases,Total Revenue
0,2.931192,179,780,2286.33


# Gender Demographics

In [229]:

df['SN'].unique()
df_unique = df.drop_duplicates(subset=['Gender', 'SN'], keep='first')
total_females = (df_unique.Gender == 'Female').sum()
total_males = (df_unique.Gender == 'Male').sum()
total_other = (df_unique.Gender == 'Other / Non-Disclosed').sum()

df_unique.Gender.value_counts()

gender_unique = pd.DataFrame(df_unique, columns = ['Gender'])
gender_unique = pd.DataFrame(df_unique['Gender'].value_counts())
gender_unique = Gender.rename(columns={'Gender': 'Total Count'})
gender_unique['Percentage of Players'] = gender_unique['Total Count']/total_players*100

gender_unique 


,Total Count,Percentage of Players
Male,465,81.151832
Female,100,17.452007
Other / Non-Disclosed,8,1.396161


# Gender Purchasing Analysis 

In [230]:

gender_demo = df.groupby('Gender').Price.agg(['mean','count','sum']).reset_index()

gender_demo = gender_demo.rename(columns={'count': 'Purchase Count'})
gender_demo = gender_demo.rename(columns={'mean': 'Average Purchase Price'})
gender_demo = gender_demo.rename(columns={'sum': 'Total Purchase Value'})

normalized_count = []

for row in gender_demo['Gender']:
    if row == 'Female':
        normalized_count.append((df_unique.Gender == 'Female').sum())
    elif row == 'Male':
        normalized_count.append((df_unique.Gender == 'Male').sum())
    else:
        normalized_count.append((df_unique.Gender == 'Other / Non-Disclosed').sum())
        
gender_demo['Normalized Count'] = normalized_count
gender_demo['Normalized Totals'] = gender_demo['Total Purchase Value']/gender_demo['Normalized Count']

gender_demo['Average Purchase Price']= gender_demo['Average Purchase Price'].map('${:,.2f}'.format)
gender_demo['Total Purchase Value']= gender_demo['Total Purchase Value'].map('${:,.2f}'.format)
gender_demo['Normalized Totals']= gender_demo['Normalized Totals'].map('${:,.2f}'.format)

gender_demo

,Gender,Average Purchase Price,Purchase Count,Total Purchase Value,Normalized Count,Normalized Totals
0,Female,$2.82,136,$382.91,100,$3.83
1,Male,$2.95,633,"$1,867.68",465,$4.02
2,Other / Non-Disclosed,$3.25,11,$35.74,8,$4.47


# Age Demographics


In [231]:
for_age = df[['Age', 'SN']].drop_duplicates()

bins = [0,9,14,19,24,29,34,39,49]
labels = ['<10','10-14', '15-19','20-24','25-29','30-34','35-39','>40',]

assert( len(bins) > len(labels))

df['age_group'] = pd.cut(df['Age'], bins, labels=labels)
for_age['age_group'] = pd.cut(for_age['Age'], bins, labels=labels)

age_out = pd.concat([for_age.age_group.value_counts(normalize=True),
                    for_age.age_group.value_counts()],
                   axis=1 )
age_out

,age_group,age_group
20-24,0.452007,259
15-19,0.174520,100
25-29,0.151832,87
30-34,0.082024,47
35-39,0.047120,27
10-14,0.040140,23
<10,0.033159,19
>40,0.019197,11


# Purchasing Analysis by Age Group

In [232]:
age_demo = df.groupby('age_group').Price.agg(['mean','count','sum']).reset_index()
age_demo.reset_index(inplace = False)
age_demo['unique_buyers'] = age_demo['age_group'].map(lambda x: age_out.to_dict()['age_group'].get(x) )
age_demo['normalized_totals'] = age_demo['sum']/age_demo['unique_buyers'].astype('float')

age_demo = age_demo.rename(columns={'count': 'Purchase Count'})
age_demo = age_demo.rename(columns={'mean': 'Average Purchase Price'})
age_demo = age_demo.rename(columns={'sum': 'Total Purchase Price'})

age_demo['Average Purchase Price']= age_demo['Average Purchase Price'].map('${:,.2f}'.format)
age_demo['Total Purchase Price']= age_demo['Total Purchase Price'].map('${:,.2f}'.format)

age_demo

C:\Users\hanna\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


,age_group,Average Purchase Price,Purchase Count,Total Purchase Price,unique_buyers,normalized_totals
0,<10,$2.98,28,$83.46,19,4.392632
1,10-14,$2.77,35,$96.95,23,4.215217
2,15-19,$2.91,133,$386.42,100,3.864200
3,20-24,$2.91,336,$978.77,259,3.779035
4,25-29,$2.96,125,$370.33,87,4.256667
5,30-34,$3.08,64,$197.25,47,4.196809
6,35-39,$2.84,42,$119.40,27,4.422222
7,>40,$3.16,17,$53.75,11,4.886364


# Top Spenders

In [233]:

top_5_spenders = df.groupby(['SN','Price']).Price.agg(['mean','count','sum']).reset_index()


top_5_spenders = top_5_spenders.rename(columns ={'count': 'Purchase Count'})
top_5_spenders = top_5_spenders.rename(columns ={'mean': 'Average Purchase Price'})
top_5_spenders = top_5_spenders.rename(columns ={'sum': 'Total Purchase Value'})

top_5_spenders['Average Purchase Price'] = top_5_spenders['Average Purchase Price'].map('${:,.2f}'.format)
top_5_spenders['Price'] = top_5_spenders['Price'].map('${:,.2f}'.format)

top_5_spenders.nlargest(5,'Total Purchase Value')

,SN,Price,Average Purchase Price,Purchase Count,Total Purchase Value
446,Lisistaya47,$3.42,$3.42,2,6.84
13,Aeliriam77,$4.95,$4.95,1,4.95
230,Frichaststa61,$4.95,$4.95,1,4.95
502,Palurrian69,$4.95,$4.95,1,4.95
531,Qiluard68,$4.95,$4.95,1,4.95


# Top Popular Items 

In [234]:

popular_items = pd.DataFrame(df.groupby(['Item Name','Item ID','Price']).Price.agg(['mean','count','sum']).reset_index())

popular_items = popular_items.rename(columns={'Price': 'Total Price'})
popular_items = popular_items.rename(columns={'count': 'Purchase Count'})
popular_items = popular_items.rename(columns={'mean': 'Average Purchase Price'})
popular_items = popular_items.rename(columns={'sum': 'Total Purchase Value'})

popular_items['Total Price'] = popular_items['Total Price'].map('${:,.2f}'.format)
popular_items['Average Purchase Price'] = popular_items['Average Purchase Price'].map('${:,.2f}'.format)
popular_items['Total Purchase Value'] = popular_items['Total Purchase Value'].map('${:,.2f}'.format)

popular_items.nlargest(5, 'Purchase Count')

,Item Name,Item ID,Total Price,Average Purchase Price,Purchase Count,Total Purchase Value
8,Arcane Gem,84,$2.23,$2.23,11,$24.53
11,"Betrayal, Whisper of Grieving Widows",39,$2.35,$2.35,11,$25.85
115,Retribution Axe,34,$4.14,$4.14,9,$37.26
122,Serenity,13,$1.49,$1.49,9,$13.41
159,Trickster,31,$2.07,$2.07,9,$18.63


# Top Profitable Items 


In [235]:

profitable_items = pd.DataFrame(df.groupby(['Item Name','Item ID','Price']).Price.agg(['mean','count','sum']).reset_index())
profitable_items = profitable_items.rename(columns={'Price': 'Total Price'})
profitable_items = profitable_items.rename(columns={'count': 'Purchase Count'})
profitable_items = profitable_items.rename(columns={'mean': 'Average Purchase Price'})
profitable_items = profitable_items.rename(columns={'sum': 'Total Purchase Value'})

profitable_items['Total Price'] = profitable_items['Total Price'].map('${:,.2f}'.format)
profitable_items['Average Purchase Price'] = profitable_items['Average Purchase Price'].map('${:,.2f}'.format)


profitable_items.nlargest(5,'Total Purchase Value')




,Item Name,Item ID,Total Price,Average Purchase Price,Purchase Count,Total Purchase Value
115,Retribution Axe,34,$4.14,$4.14,9,37.26
135,Spectral Diamond Doomblade,115,$4.25,$4.25,7,29.75
98,Orenmir,32,$4.95,$4.95,6,29.70
127,Singed Scalpel,103,$4.87,$4.87,6,29.22
137,"Splitter, Foe Of Subtlety",107,$3.61,$3.61,8,28.88
